In [73]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, img_to_array
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sqlalchemy import create_engine, Column, Integer, String, LargeBinary
from sqlalchemy.orm import sessionmaker, declarative_base
import os
from sqlalchemy.orm import declarative_base

In [8]:
train_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\trenyravimas"
val_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\validacija"
test_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys\testas"

In [74]:

Bazine_klase = declarative_base()
rysys_su_baze = create_engine('sqlite:///pomidoru_lapai.db')

Session = sessionmaker(bind=rysys_su_baze)
sesija = Session()

In [10]:
class PomidoraiTrenyravimas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_trenyravimo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)


In [11]:
def irasyti_trenyravimo_paveikslelius(sesija, trenyravimo_kelias):

    for klases_pavadinimas in os.listdir(trenyravimo_kelias):
        klases_kelias = os.path.join(trenyravimo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTrenyravimas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTrenyravimas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti trenyravimo paveiksleiai")

In [12]:
class PomidoraiValidacija(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_validacijos_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_validacijos_paveikslelius(sesija, validacijos_kelias):

    for klases_pavadinimas in os.listdir(validacijos_kelias):
        klases_kelias = os.path.join(validacijos_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiValidacija).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiValidacija(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti validacijos paveiksleiai")

In [ ]:
class PomidoraiTestas(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_testo_duomenys'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

def irasyti_testo_paveikslelius(sesija, testo_kelias):

    for klases_pavadinimas in os.listdir(testo_kelias):
        klases_kelias = os.path.join(testo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTestas).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTestas(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti testo paveiksleiai")

In [14]:
Bazine_klase.metadata.create_all(rysys_su_baze)

irasyti_trenyravimo_paveikslelius(sesija,train_kelias)
irasyti_validacijos_paveikslelius(sesija,val_kelias)
irasyti_testo_paveikslelius(sesija,test_kelias)

Irasyti trenyravimo paveiksleiai
Irasyti validacijos paveiksleiai
Irasyti testo paveiksleiai


In [15]:

train_df = pd.read_sql_table('Pomidoru_lapai_trenyravimo_duomenys', con=rysys_su_baze)

val_df = pd.read_sql_table('Pomidoru_lapai_validacijos_duomenys', con=rysys_su_baze)

test_df = pd.read_sql_table('Pomidoru_lapai_testo_duomenys', con=rysys_su_baze)


# print(train_df.head())
# print(val_df.head())
# print(test_df.head())

In [16]:
def issitraukti_paveikslelius(df, dydis=(224, 224)):
    paveiksleliai = []
    klasifikacijos = []

    for indeksas, eilute in df.iterrows():
        paveikslelis = load_img(eilute['kelias'], target_size=dydis)
        paveikslelis_array = img_to_array(paveikslelis)
        paveiksleliai.append(paveikslelis_array)
        klasifikacijos.append(eilute['klases_pavadinimas'])

    return np.array(paveiksleliai), np.array(klasifikacijos)

x_train, y_train = issitraukti_paveikslelius(train_df)
x_val, y_val = issitraukti_paveikslelius(val_df)
x_test, y_test = issitraukti_paveikslelius(test_df)

In [17]:
def uzkoduoti_klases_lable(y_train, y_val, y_test):

    enkoderis = LabelEncoder()
    y_train_skaiciais = enkoderis.fit_transform(y_train)
    y_val_skaiciais = enkoderis.transform(y_val)
    y_test_skaiciais = enkoderis.transform(y_test)

    print("Klases uzkoduotos")

    return y_train_skaiciais,y_val_skaiciais,y_test_skaiciais

y_train, y_val, y_test = uzkoduoti_klases_lable(y_train,y_val,y_test)

Klases uzkoduotos


HSV paveiksleliai
.


In [54]:
import os
import cv2
import numpy as np
from PIL import Image
import tensorflow as tf

In [ ]:
def konvertuoti_i_hsv(pav_failo_kelias):
    paveikslelis = cv2.imread(pav_failo_kelias)
    paveikslelis = cv2.cvtColor(paveikslelis, cv2.COLOR_BGR2RGB)
    hsv_paveikslelis = cv2.cvtColor(paveikslelis, cv2.COLOR_RGB2HSV)
    return hsv_paveikslelis

In [ ]:
def issaugoti_hsv_paveiksleli(hsv_masyvas, issaugojimo_kelias):
    rgb_masyvas = cv2.cvtColor(hsv_masyvas, cv2.COLOR_HSV2RGB)
    paveikslelis = Image.fromarray(rgb_masyvas) #sukuriamas jpg. objektas is masyvo
    paveikslelis.save(issaugojimo_kelias)

In [ ]:
def konvertuoti_viska_i_hsv(pradinis_kelias, issaugojimo_kelias):
    for aplanko_kelias in ["trenyravimas", "validacija", "testas"]:
        pilno_aplanko_kelias = os.path.join(pradinis_kelias, aplanko_kelias)
        naujo_aplanko_kelias = os.path.join(issaugojimo_kelias, aplanko_kelias)

        for klases_kelias in os.listdir(pilno_aplanko_kelias):
            pradzios_klases_aplanko_kelias= os.path.join(pilno_aplanko_kelias, klases_kelias)
            naujos_klases_aplanko_kelias = os.path.join(naujo_aplanko_kelias, klases_kelias)
            os.makedirs(naujos_klases_aplanko_kelias, exist_ok=True) # jei aplankas jau sukurtas eis prie kito veiksmo jei nera sukurs 

            for failo_pavadinimo_kelias in os.listdir(pradzios_klases_aplanko_kelias):
                if failo_pavadinimo_kelias.lower().endswith(('.jpg', '.png', '.jpeg')):
                    pilnas_failo_kelias = os.path.join(pradzios_klases_aplanko_kelias, failo_pavadinimo_kelias)
                    hsv_paveikslelis = konvertuoti_i_hsv(pilnas_failo_kelias)
                    pilnas_kelias_issaugojimui = os.path.join(naujos_klases_aplanko_kelias, failo_pavadinimo_kelias)
                    issaugoti_hsv_paveiksleli(hsv_paveikslelis, pilnas_kelias_issaugojimui)

In [58]:
pradinis_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys"
issaugojimo_kelias = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys_hsv"
konvertuoti_viska_i_hsv(pradinis_kelias, issaugojimo_kelias)

HSV i duomenu baze 
.


In [75]:
class PomidoraiTrenyravimasHSV(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_trenyravimo_hsv'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

In [76]:
class PomidoraiValidacijaHSV(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_validacijos_hsv'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

In [77]:
class PomidoraiTestasHSV(Bazine_klase):

    __tablename__ = 'Pomidoru_lapai_testo_hsv'

    id = Column(Integer,primary_key = True)
    kelias = Column(String, unique=True, nullable=False)  
    klases_pavadinimas = Column(String, nullable=False)

In [79]:
def irasyti_trenyravimo_paveikslelius_HSV(sesija, trenyravimo_kelias):

    for klases_pavadinimas in os.listdir(trenyravimo_kelias):
        klases_kelias = os.path.join(trenyravimo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTrenyravimasHSV).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTrenyravimasHSV(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti treniravimo HSV paveiksleiai")

In [80]:
def irasyti_validacijos_paveikslelius_HSV(sesija, validacijos_kelias):

    for klases_pavadinimas in os.listdir(validacijos_kelias):
        klases_kelias = os.path.join(validacijos_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiValidacijaHSV).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiValidacijaHSV(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti validacijos HSV paveiksleiai")

In [81]:
def irasyti_testo_paveikslelius_HSV(sesija, testo_kelias):

    for klases_pavadinimas in os.listdir(testo_kelias):
        klases_kelias = os.path.join(testo_kelias,klases_pavadinimas)

        if os.path.isdir(klases_kelias):
            for paveikslelio_pavadinimas in os.listdir(klases_kelias):
                paveikslelio_kelias = os.path.join(klases_kelias,paveikslelio_pavadinimas)

                egzistuoja = sesija.query(PomidoraiTestasHSV).filter_by(kelias=paveikslelio_kelias).first()

                if not egzistuoja:

                    naujas_irasas = PomidoraiTestasHSV(kelias = paveikslelio_kelias, klases_pavadinimas = klases_pavadinimas)

                    sesija.add(naujas_irasas)

    sesija.commit()

    print("Irasyti testo HSV paveiksleiai")

In [82]:
train_kelias_HSV = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys_hsv\trenyravimas"
val_kelias_HSV = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys_hsv\validacija"
test_kelias_HSV = r"C:\Users\Vartotojas\Desktop\POMIDORAI\pomidoru_duomenys_hsv\testas"

Bazine_klase.metadata.create_all(rysys_su_baze)

irasyti_trenyravimo_paveikslelius_HSV(sesija,train_kelias_HSV)
irasyti_validacijos_paveikslelius_HSV(sesija,val_kelias_HSV)
irasyti_testo_paveikslelius_HSV(sesija,test_kelias_HSV)

Irasyti treniravimo HSV paveiksleiai
Irasyti validacijos HSV paveiksleiai
Irasyti testo HSV paveiksleiai


CNN modelis su HSV
.

In [84]:

train_df_hsv = pd.read_sql_table('Pomidoru_lapai_trenyravimo_hsv', con=rysys_su_baze)

val_df_hsv = pd.read_sql_table('Pomidoru_lapai_validacijos_hsv', con=rysys_su_baze)

test_df_hsv = pd.read_sql_table('Pomidoru_lapai_testo_hsv', con=rysys_su_baze)

In [85]:
def issitraukti_paveikslelius(df, dydis=(128, 128)):
    paveiksleliai = []
    klasifikacijos = []

    for indeksas, eilute in df.iterrows():
        paveikslelis = load_img(eilute['kelias'], target_size=dydis)
        paveikslelis_array = img_to_array(paveikslelis)
        paveiksleliai.append(paveikslelis_array)
        klasifikacijos.append(eilute['klases_pavadinimas'])

    return np.array(paveiksleliai), np.array(klasifikacijos)

x_train_hsv, y_train_hsv = issitraukti_paveikslelius(train_df_hsv)
x_val_hsv, y_val_hsv = issitraukti_paveikslelius(val_df_hsv)
x_test_hsv, y_test_hsv = issitraukti_paveikslelius(test_df_hsv)

In [86]:
def uzkoduoti_klases_lable(y_train, y_val, y_test):

    enkoderis = LabelEncoder()
    y_train_skaiciais = enkoderis.fit_transform(y_train)
    y_val_skaiciais = enkoderis.transform(y_val)
    y_test_skaiciais = enkoderis.transform(y_test)

    print("Klases uzkoduotos")

    return y_train_skaiciais,y_val_skaiciais,y_test_skaiciais

y_train_hsv, y_val_hsv, y_test_hsv = uzkoduoti_klases_lable(y_train_hsv,y_val_hsv,y_test_hsv)

Klases uzkoduotos


Modelis

In [93]:
from tensorflow.keras.utils import to_categorical

y_train_hsv_kat = to_categorical(y_train_hsv)
y_val_hsv_kat = to_categorical(y_val_hsv)
y_test_hsv_kat = to_categorical(y_test_hsv)

In [94]:
x_train_hsv = x_train_hsv.astype("float32") / 255.0
x_val_hsv = x_val_hsv.astype("float32") / 255.0
x_test_hsv = x_test_hsv.astype("float32") / 255.0

In [95]:
x_train = x_train_hsv
x_val = x_val_hsv
x_test = x_test_hsv
y_train = y_train_hsv_kat
y_val = y_val_hsv_kat
y_test = y_test_hsv_kat

In [96]:
klasiu_skaicius = y_train.shape[1]
print(klasiu_skaicius)
ivesties_forma = (128,128,3)
imties_dydis = 32
SEED = 42

6


In [ ]:
def sukurti_cnn_hsv(ivesties_forma, klasiu_skaicius, learning_rate=0.0001):

    modelis = keras.Sequential([
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=ivesties_forma),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),

        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.4),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),
        layers.Dense(klasiu_skaicius, activation='softmax')])

    optimizieris = keras.optimizers.Adam(learning_rate=learning_rate)

    modelis.compile(optimizer=optimizieris, loss='categorical_crossentropy', metrics=['accuracy'])

    return modelis

modelis = sukurti_cnn_hsv(ivesties_forma, klasiu_skaicius)

c:\Users\Vartotojas\Desktop\POMIDORAI\pomidorai_repositorija\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

sustabdymas = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = modelis.fit(x_train,y_train, validation_data=(x_val,y_val), epochs=40, callbacks=[sustabdymas])

nuostolis, tikslumas = modelis.evaluate(x_test,y_test, verbose=1)

print(f"Testo nuostolis loss: {nuostolis:.4f}")
print(f"Testo tikslumas accuracy: {tikslumas:.4f}")

Epoch 1/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 142s 950ms/step - accuracy: 0.4959 - loss: 6.7970 - val_accuracy: 0.1667 - val_loss: 41.2473
Epoch 2/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 124s 822ms/step - accuracy: 0.6601 - loss: 1.0428 - val_accuracy: 0.1744 - val_loss: 57.5892
Epoch 3/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 116s 821ms/step - accuracy: 0.7017 - loss: 0.8466 - val_accuracy: 0.2922 - val_loss: 19.3102
Epoch 4/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 143s 826ms/step - accuracy: 0.7080 - loss: 0.8343 - val_accuracy: 0.2689 - val_loss: 8.1172
Epoch 5/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 142s 824ms/step - accuracy: 0.7100 - loss: 0.8163 - val_accuracy: 0.6756 - val_loss: 1.4330
Epoch 6/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 117s 830ms/step - accuracy: 0.7403 - loss: 0.6834 - val_accuracy: 0.4656 - val_loss: 4.3017
Epoch 7/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 127s 900ms/step - accuracy: 0.7908 - loss: 0.6452 - val_accuracy: 0.6878 - val_loss: 2.6252
Epoch 8/40
141/141 ━━━━━━━━━━━━━━━━━━━━ 116s 822ms/step - accuracy: 0.823

In [99]:
modelis.save("hsv_cnn_modelis.h5")